Szablon do 'eksperymentalnego' sprawdzania hipotezy (szukania przykładu, który by ją obalił)  na temat uczenia się splotów używająć SRS.

In [1]:
import copy, sys
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
from importlib import reload

import inferring.InferringDFA as InferringDFA
import utils.automats.DFA.PatternDFA as PatternDFA
import utils.advice_systems.SRSconv as SRSconv

reload(SRSconv)
reload(InferringDFA)
reload(PatternDFA)

from inferring.InferringDFA import InferringDFA
from utils.automats.DFA.PatternDFA import PatternDFA
from utils.advice_systems.SRSconv import SRSconv

In [2]:
def run_learning_process(test, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                            advice_system, 
                            check_consistency=check_consistency, 
                            equiv_query_fashion=equiv_query_fashion,
                            debug=debug)

    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return cnt, copy.deepcopy(dfa) 

In [3]:
class test_example():
    def __init__(self, type, dfa, description=''):
        self.type = type
        self.dfa = copy.deepcopy(dfa)
        self.description = description

In [4]:
import random 
def random_patterns(input_signs, n=5, min_length=1, max_length=10):
    patterns = set()
    while len(patterns) < n:
        p = ''.join(random.choices(population=input_signs,k=random.randint(min_length, max_length)))
        patterns.add(p)
    return list(patterns)

In [32]:
def create_PatternDetection_test(type1="AND", type2="AND", in_alphabet=[], max_number_of_patterns=6, min_length=1, max_length=10):
    m = [min_length, max_length]
    types = [type1, type2]

    dfas, ps = [], []
    for i in range(2):
        n = random.randint(1,max_number_of_patterns)
        if types[i]=="AND":
            ps.append(random_patterns(in_alphabet, n=n, min_length=max(1,m[0]), max_length=m[1]))
        else:
            ps.append(random_patterns(in_alphabet, n=n, min_length=m[0], max_length=m[1]))

        dfa_ = PatternDFA(input_signs=in_alphabet, patterns=ps[i], _type=types[i])
        dfas.append(dfa_)

    return copy.deepcopy(dfas[0]), ps[0], copy.deepcopy(dfas[1]), ps[1]

In [63]:
class Result():
    def __init__(self, ps1, ps2, cnt_dfs, cnt_bfs, upper_bound):
        self.ps = [ps1, ps2]
        self.cnt = (cnt_dfs, cnt_bfs)
        self.upper_bound = upper_bound
    
    # def __str__(self):
    #     "cnt = " + str(self.cnt) + " upper_bound = " + str(self.upper_bound) + ""
    
    def print_it(self):
        print(f"cnt = {self.cnt} upper_bound = {self.upper_bound}")
    
    def get_cnt_dfs(self):
        return self.cnt[0]
    
    def get_cnt_bfs(self):
        return self.cnt[1]
    

In [97]:
def do_tests():
    results = []
    for i in range(200):
        
        alph = ['a', 'b', 'c']
        dfa1, ps1, dfa2, ps2 = create_PatternDetection_test(type1='OR', type2='OR', in_alphabet=alph, max_number_of_patterns=5, min_length=0, max_length=10)

        cnt1, d1 = run_learning_process(test=test_example(type="dfa", dfa=dfa1))
        cnt2, d2 = run_learning_process(test=test_example(type="dfa", dfa=dfa2))
        convdfa_ = convDFA(type="conv", dfa1=d1, dfa2=d2)

        cnt_dfs, d   = run_learning_process(test=test_example(type="conv", dfa=convdfa_), advice_system=SRSconv(), check_consistency=True, equiv_query_fashion="DFS")
        cnt_bfs, d   = run_learning_process(test=test_example(type="conv", dfa=convdfa_), advice_system=SRSconv(), check_consistency=True, equiv_query_fashion="BFS")

        upper_bound = cnt1[1] + cnt2[1]
        print(f"{i}: cnt_dfs = {cnt_dfs[1]}, cnt_bfs = {cnt_bfs[1]}, upper_bound = {upper_bound}")
        assert cnt_dfs[1] <= upper_bound and cnt_bfs[1] <= upper_bound, "hipoteza klap" 
        results.append(Result(ps1=ps1, ps2=ps2, cnt_dfs=cnt_dfs[1], cnt_bfs=cnt_bfs[1], upper_bound=upper_bound))

    return results

In [98]:
res = do_tests()

0: cnt_dfs = 1, cnt_bfs = 1, upper_bound = 2
1: cnt_dfs = 3, cnt_bfs = 3, upper_bound = 4
2: cnt_dfs = 4, cnt_bfs = 4, upper_bound = 6
3: cnt_dfs = 3, cnt_bfs = 3, upper_bound = 3
4: cnt_dfs = 5, cnt_bfs = 5, upper_bound = 7
5: cnt_dfs = 2, cnt_bfs = 2, upper_bound = 3
6: cnt_dfs = 2, cnt_bfs = 2, upper_bound = 2
7: cnt_dfs = 5, cnt_bfs = 5, upper_bound = 6
8: cnt_dfs = 2, cnt_bfs = 3, upper_bound = 5
9: cnt_dfs = 5, cnt_bfs = 6, upper_bound = 7
10: cnt_dfs = 2, cnt_bfs = 2, upper_bound = 3
11: cnt_dfs = 2, cnt_bfs = 3, upper_bound = 3
12: cnt_dfs = 5, cnt_bfs = 5, upper_bound = 7
13: cnt_dfs = 2, cnt_bfs = 2, upper_bound = 3
14: cnt_dfs = 5, cnt_bfs = 9, upper_bound = 9
15: cnt_dfs = 1, cnt_bfs = 1, upper_bound = 2
16: cnt_dfs = 4, cnt_bfs = 4, upper_bound = 5
17: cnt_dfs = 3, cnt_bfs = 6, upper_bound = 7
18: cnt_dfs = 4, cnt_bfs = 4, upper_bound = 5
19: cnt_dfs = 1, cnt_bfs = 1, upper_bound = 2
20: cnt_dfs = 7, cnt_bfs = 9, upper_bound = 10
21: cnt_dfs = 1, cnt_bfs = 1, upper_bound =

AssertionError: hipoteza klap

In [105]:
run_learning_process(test=test_example(type="dfa", dfa=d1))
run_learning_process(test=test_example(type="dfa", dfa=d2))
# 11 vs 9 

print(upper_bound, cnt_dfs, cnt_bfs)

kontrprzykłady: [11, 11, 11, 11, 11], cnt = [4517, 6]
kontrprzykłady: [10, 10, 11, 11], cnt = [3471, 5]


NameError: name 'upper_bound' is not defined

In [100]:
d1.print_transitions()
d2.print_transitions()

(0,a) --> 0
(0,b) --> 1
(0,c) --> 19
(0,d) --> 31
(1,a) --> 0
(1,b) --> 2
(1,c) --> 19
(1,d) --> 31
(2,a) --> 0
(2,b) --> 3
(2,c) --> 19
(2,d) --> 31
(3,a) --> 3
(3,b) --> 3
(3,c) --> 4
(3,d) --> 12
(4,a) --> 3
(4,b) --> 3
(4,c) --> 5
(4,d) --> 12
(5,a) --> 6
(5,b) --> 3
(5,c) --> 5
(5,d) --> 12
(6,a) --> 3
(6,b) --> 3
(6,c) --> 7
(6,d) --> 12
(7,a) --> 7
(7,b) --> 7
(7,c) --> 7
(7,d) --> 8
(8,a) --> 7
(8,b) --> 7
(8,c) --> 7
(8,d) --> 9
(9,a) --> 7
(9,b) --> 7
(9,c) --> 7
(9,d) --> 10
(10,a) --> 7
(10,b) --> 7
(10,c) --> 7
(10,d) --> 11
(11,a) --> 11
(11,b) --> 11
(11,c) --> 11
(11,d) --> 11
(12,a) --> 3
(12,b) --> 3
(12,c) --> 4
(12,d) --> 13
(13,a) --> 3
(13,b) --> 3
(13,c) --> 4
(13,d) --> 14
(14,a) --> 3
(14,b) --> 3
(14,c) --> 4
(14,d) --> 15
(15,a) --> 15
(15,b) --> 15
(15,c) --> 16
(15,d) --> 15
(16,a) --> 15
(16,b) --> 15
(16,c) --> 17
(16,d) --> 15
(17,a) --> 18
(17,b) --> 15
(17,c) --> 17
(17,d) --> 15
(18,a) --> 15
(18,b) --> 15
(18,c) --> 11
(18,d) --> 15
(19,a) --> 0
(19,

In [ ]:
for r in res:
    rank = r.upper_bound - max(r.get_cnt_dfs(), r.get_cnt_bfs())
    r.print_it()
    print(f"rank = {rank}")
res.sort(key= lambda r : r.upper_bound-max(r.get_cnt_dfs(), r.get_cnt_bfs()))

In [87]:
k = 0 
for r in res:
    if r.get_cnt_dfs() > r.get_cnt_bfs():
        r.print_it()
        k += 1 
print(k)

cnt = (5, 4) upper_bound = 5
cnt = (4, 3) upper_bound = 5
2


In [92]:
dfa1 = PatternDFA(input_signs=['a', 'b', 'c', 'd'], patterns=['ccac', 'bbb', 'dddd'], _type="AND")
dfa2 = PatternDFA(input_signs=['a', 'b', 'c', 'd'], patterns=['caacc', 'ccc', 'aaaa'], _type="AND")

cnt1, d1 = run_learning_process(test=test_example(type="dfa", dfa=dfa1))
cnt2, d2 = run_learning_process(test=test_example(type="dfa", dfa=dfa2))

In [94]:
print(d1.Q, d2.Q)

40 42


In [95]:
convdfa_12 = convDFA(type="conv", dfa1=d1, dfa2=d2)
print(convdfa_12.Q)
cnt_dfs, d = run_learning_process(test=test_example(type="conv", dfa=convdfa_12), advice_system=SRSconv(), check_consistency=True, equiv_query_fashion="DFS")
cnt_bfs, d = run_learning_process(test=test_example(type="conv", dfa=convdfa_12), advice_system=SRSconv(), check_consistency=True, equiv_query_fashion="BFS")

1680


In [96]:
print(f"{cnt1}, {cnt2}, {cnt_dfs}, {cnt_bfs}")

[4517, 6], [3471, 5], [2754089, 10], [1970652, 9]
